In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
#from pandas_profiling import ProfileReport
#% matpllotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import seaborn as sns

In [2]:
from pandas_profiling import ProfileReport

In [3]:
df = pd.read_csv('Admission_Prediction.csv')

In [5]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [7]:
pf = ProfileReport(df)
pf.to_widgets()

TypeError: concat() got an unexpected keyword argument 'join_axes'

In [8]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [17]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].mean())

In [24]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [25]:
#'Serial No.' coluumn is not significant to dataset, so we will drop it
df.drop(columns= ['Serial No.'],inplace = True)

In [26]:
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,316.558763,107.187755,3.121649,4.5,4.5,9.65,1,0.92
1,316.558763,107.187755,3.121649,4.0,4.5,8.87,1,0.76
2,316.558763,107.187755,3.121649,3.0,3.5,8.00,1,0.72
3,316.558763,107.187755,3.121649,3.5,2.5,8.67,1,0.80
4,316.558763,107.187755,3.121649,2.0,3.0,8.21,0,0.65


In [28]:
# Split features and labels
X = df.drop(columns = ['Chance of Admit'])
y = df['Chance of Admit']


In [29]:
X.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,316.558763,107.187755,3.121649,4.5,4.5,9.65,1
1,316.558763,107.187755,3.121649,4.0,4.5,8.87,1
2,316.558763,107.187755,3.121649,3.0,3.5,8.00,1
3,316.558763,107.187755,3.121649,3.5,2.5,8.67,1
4,316.558763,107.187755,3.121649,2.0,3.0,8.21,0


In [30]:
y.head()

0    0.92
1    0.76
2    0.72
3    0.80
4    0.65
Name: Chance of Admit, dtype: float64

In [32]:
#Since GRE Score and TOEFL Score value is higher wrt others features we will use StandarScaler
st_sc = StandardScaler()
arr = st_sc.fit_transform(X)

In [34]:
pd.DataFrame(arr)

,0,1,2,3,4,5,6
0,0.0,-2.842171e-14,-4.440892e-16,1.137360,1.098944,1.776806,0.886405
1,0.0,-2.842171e-14,-4.440892e-16,0.632315,1.098944,0.485859,0.886405
2,0.0,-2.842171e-14,-4.440892e-16,-0.377773,0.017306,-0.954043,0.886405
3,0.0,-2.842171e-14,-4.440892e-16,0.127271,-1.064332,0.154847,0.886405
4,0.0,-2.842171e-14,-4.440892e-16,-1.387862,-0.523513,-0.606480,-1.128152
...,...,...,...,...,...,...,...
495,0.0,-2.842171e-14,-4.440892e-16,1.137360,0.558125,0.734118,0.886405
496,0.0,-2.842171e-14,-4.440892e-16,1.642404,1.639763,2.140919,0.886405
497,0.0,-2.842171e-14,-4.440892e-16,1.137360,1.639763,1.627851,0.886405
498,0.0,-2.842171e-14,-4.440892e-16,0.632315,1.639763,-0.242367,-1.128152


In [42]:
#now we are checking multi collinearity 
from statsmodels.stats.outliers_influence import variance_inflation_factor

df2 = pd.DataFrame()
df2['Vif'] = [variance_inflation_factor(arr,i) for i in range(arr.shape[1])]
df2

#Since there is no value greater than 10 it means no multicollinearity

,Vif
0,NaN
1,0.000000
2,0.000000
3,2.396198
4,1.987793
5,2.473945
6,1.348696


In [44]:
# Splitting 90% data for training and 10% data for test)
X_train, X_test, y_train, y_test = train_test_split(arr, y , test_size= .10, random_state= 100)

In [46]:
#Applying LR Model
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [47]:
lr.coef_

array([ 0.00000000e+00, -6.93889390e-18,  0.00000000e+00,  4.70644171e-03,
        1.68983646e-02,  1.02295660e-01,  1.80579362e-02])

In [48]:
lr.intercept_

0.719617961513244

In [49]:
#physically saving the model
pickle.dump(lr,open('admission_pred_mo.pickle','wb'))

In [50]:
X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,316.558763,107.187755,3.121649,4.5,4.5,9.65,1
1,316.558763,107.187755,3.121649,4.0,4.5,8.87,1
2,316.558763,107.187755,3.121649,3.0,3.5,8.00,1
3,316.558763,107.187755,3.121649,3.5,2.5,8.67,1
4,316.558763,107.187755,3.121649,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,316.558763,107.187755,3.121649,4.5,4.0,9.02,1
496,316.558763,107.187755,3.121649,5.0,5.0,9.87,1
497,316.558763,107.187755,3.121649,4.5,5.0,9.56,1
498,316.558763,107.187755,3.121649,4.0,5.0,8.43,0


In [53]:
lr.predict(st_sc.transform([[316.558763, 107.187755, 3.12164, 4.5, 4.0, 9.02,1]]))

C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([0.82550606])

In [52]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [54]:
#now we will check accuracy
lr.score(X_test, y_test)

0.8203326265719136

In [57]:
#Applying Lasso Regularization to adjust loss function
lassocv = LassoCV(alphas=None, cv=50, max_iter=100, normalize=True)
lassocv.fit(X_train, y_train)

C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


LassoCV(cv=50, max_iter=100, normalize=True)

In [59]:
lassocv.alpha_

5.84571056207548e-06

In [60]:
la_mo = Lasso(alpha=lassocv.alpha_)
la_mo.fit(X_train,y_train)

Lasso(alpha=5.84571056207548e-06)

In [61]:
la_mo.score(X_test, y_test)

0.8203396042393705

In [66]:
#Applying Ridge Regularization to adjust loss function
ridge = RidgeCV(alphas=np.random.uniform(0,10,50), cv= 50, normalize=True)
ridge.fit(X_train,y_train)


C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:



C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:



C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:



C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:



C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:



...

C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:



C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:



C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:



C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\ankussingh\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:



RidgeCV(alphas=array([1.41303985, 5.83975276, 6.75314813, 4.63614593, 6.13274739,
       8.39045397, 0.97200083, 8.70238895, 8.35183892, 3.83480822,
       5.71857679, 8.96732549, 5.25188709, 8.79579317, 6.19069121,
       3.81459394, 9.7691766 , 7.81342904, 3.91761767, 0.23199897,
       2.62654976, 6.61013299, 9.86305836, 8.99946135, 9.4986875 ,
       6.15918066, 9.12666234, 7.8411377 , 6.71340948, 8.64812941,
       7.49305797, 1.14109954, 0.40321203, 2.97295941, 1.24717844,
       0.7512228 , 6.8621555 , 9.6075564 , 3.23433778, 7.49292989,
       0.02109222, 5.36093807, 7.60439126, 4.27237038, 0.02178509,
       9.71933866, 5.56384791, 5.50805988, 4.6704835 , 4.48839494]),
        cv=50, normalize=True)

In [67]:
ridge.alpha_

0.02178508686904057

In [68]:
ri_mo = Ridge(ridge.alpha_)
ri_mo.fit(X_train, y_train)

Ridge(alpha=0.02178508686904057)

In [69]:
ri_mo.score(X_test,y_test)

0.8203437361821218

In [70]:
#Elastic Net refer 19th Sept ML DL Playlist